# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation. To run this example Julia's `PyCall` package
needs to be able to find an installation of `pymatgen`.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

┌ Warning: `vendor()` is deprecated, use `BLAS.get_config()` and inspect the output instead
│   caller = npyinitialize() at numpy.jl:67
└ @ PyCall /home/runner/.julia/packages/PyCall/L0fLP/src/numpy.jl:67


Setup an LDA model and discretize using
a single k-point and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=(1, 1, 1))

PlaneWaveBasis discretization:
    Ecut                 : 5.0 Ha
    fft_size             : (32, 32, 32), 32768 total points
    kgrid type           : Monkhorst-Pack
    kgrid                : [1, 1, 1]
    num. irred. kpoints  : 1

    Discretized Model(lda_xc_teter93, 3D):
        lattice (in Bohr)    : [0         , 10.2631   , 10.2631   ]
                               [10.2631   , 0         , 10.2631   ]
                               [10.2631   , 10.2631   , 0         ]
        unit cell volume     : 2162.1 Bohr³
    
        atoms                : Si₁₆
        atom potentials      : ElementPsp(Si, psp="hgh/lda/si-q4")
    
        num. electrons       : 64
        spin polarization    : none
        temperature          : 0 Ha
    
        terms                : Kinetic()
                               AtomicLocal()
                               AtomicNonlocal()
                               Ewald()
                               PspCorrection()
                               

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.126937e+02     1.464220e+00
 * time: 0.6401729583740234
     1     1.122035e+01     7.334003e-01
 * time: 1.4844939708709717
     2    -1.195517e+01     8.549777e-01
 * time: 1.724377155303955
     3    -3.411999e+01     6.834433e-01
 * time: 1.8495640754699707
     4    -4.762319e+01     5.702326e-01
 * time: 1.9764800071716309
     5    -5.692044e+01     2.238606e-01
 * time: 2.103924036026001
     6    -5.984010e+01     1.174792e-01
 * time: 2.2050399780273438
     7    -6.091982e+01     5.429760e-02
 * time: 2.828202962875366
     8    -6.129743e+01     7.226635e-02
 * time: 2.9039180278778076
     9    -6.158072e+01     3.469794e-02
 * time: 2.987352132797241
    10    -6.176809e+01     2.748429e-02
 * time: 3.0701589584350586
    11    -6.192700e+01     1.989239e-02
 * time: 3.1582961082458496
    12    -6.199515e+01     1.760090e-02
 * time: 3.249221086502075
    13    -6.207155e+01     1.494827e-02
 * time: 3.629390001296997

In [5]:
scfres.energies

Energy breakdown (in Ha):
    Kinetic             25.7671057
    AtomicLocal         -18.8557693
    AtomicNonlocal      14.8522672
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485370 
    Xc                  -19.3336819

    total               -62.261666454521